In [1]:
# Fetch transcript from Youtube video

import json
from fetch_transcript.youtube_fetcher import YouTubeTranscriptFetcher


video_id = "Rwqx4UbKjR8"

fetcher = YouTubeTranscriptFetcher(languages=["en", "vi"])
result = fetcher.fetch(video_id)

print(json.dumps(
    {
        "video_id": result["video_id"],
        "language": result["language"],
        "duration": result["duration"],
        "cleaned_text": result["text"]
    },
    indent=2,
    ensure_ascii=False
))



{
  "video_id": "Rwqx4UbKjR8",
  "language": "English (auto-generated)",
  "duration": {
    "seconds": 3898.8,
    "minutes": 64.98
  },
  "cleaned_text": "[9.76s] hello everyone leo here ready to embark [12.72s] on a journey into the world of dreams [15.76s] not the sleepy time ones but those [18.12s] dreams that get you up in the morning [20.48s] and i'm gwen here to explore how our [23.32s] passions shape our lives [33.56s] so leo let's define dreams in the [36.40s] context of today's [38.12s] chat right gwen when we talk about [41.00s] dreams here we're talking about those [42.96s] big goals and aspirations the things [45.28s] that make us excited about the future [48.08s] like someone dreaming of becoming an [49.72s] artist an engineer or mastering a new [52.56s] skill it's all about what you're [55.04s] passionate about exactly and my dream [58.36s] it's to bring joy to people whether it's [61.56s] through comedy acting or just being my [64.40s] charming self i mean who wouldn't

In [2]:
# Token count estimation
from utils.token_counter import estimate_tokens
token_count = estimate_tokens(result["text"])
print(f"Estimated token count: {token_count}")

Estimated token count: 15180


In [3]:
# Router to choose long flow or short flow based on token count

from pipeline.long_flow import run_long_flow
from pipeline.short_flow import run_short_flow
import json
transcript = result["text"]
token_count = len(transcript.split())
language = result["language"]
if token_count > 1500:
    output = run_long_flow(transcript, language)
else:
    output = run_short_flow(transcript, language)
print(json.dumps(output.model_dump(), indent=2, ensure_ascii=False))

[
  {
    "section_id": 1,
    "title": "Defining Dreams and Their Importance",
    "start": 10.0,
    "end": 148.0,
    "summary": "This section defines dreams as significant goals and aspirations that ignite excitement for the future, encompassing ambitions like becoming an artist, engineer, or mastering a new skill, all stemming from passion. The speakers share their personal dreams: Leo aims to bring joy through various performance arts, while Gwen desires to make a difference through writing, teaching, or environmental advocacy. They emphasize that dreams are not solely about the destination but also about the enjoyment and aliveness experienced during the pursuit. Learning English is presented as a dream that opens doors to opportunities, friendships, and cultural understanding. The speakers highlight that pursuing dreams requires courage and commitment, pushing individuals beyond their comfort zones and making life more exciting and rewarding. They encourage listeners to value t

In [4]:
# Generate outline from transcript

from llm.gemini_client import GeminiClient
from llm.prompts import build_outline_prompt, OUTLINE_PROMPT
from schemas.output_format import OutlineOutput
import json

video_transcript = result["text"]
prompt = build_outline_prompt(video_transcript=video_transcript)
gemini = GeminiClient()
response = gemini.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_json_schema": OutlineOutput.model_json_schema(),
    }
)
outline = OutlineOutput.model_validate_json(response.text)

print("===== OUTLINE INFORMATION =====")
print(f"Total sections: {len(outline.sections)}")
print(json.dumps(outline.model_dump(), indent=2, ensure_ascii=False))

===== OUTLINE INFORMATION =====
Total sections: 38
{
  "sections": [
    {
      "section_id": 1,
      "title": "Introduction to Dreams as Aspirations",
      "start": 0.0,
      "end": 33.0,
      "keywords": [
        "welcome",
        "dreams",
        "aspirations",
        "passions"
      ]
    },
    {
      "section_id": 2,
      "title": "Defining and Pursuing Dreams",
      "start": 33.0,
      "end": 148.0,
      "keywords": [
        "goals",
        "aspirations",
        "passion",
        "courage",
        "commitment"
      ]
    },
    {
      "section_id": 3,
      "title": "Key Terms and Concepts of Dreams",
      "start": 148.0,
      "end": 266.0,
      "keywords": [
        "vocabulary",
        "aspirations",
        "advocating",
        "unlock",
        "commitment"
      ]
    },
    {
      "section_id": 4,
      "title": "Practical Steps for Learning English",
      "start": 266.0,
      "end": 387.0,
      "keywords": [
        "learning english",
     

In [5]:
# Segment transcript according to outline


from preprocess.segmenter import TranscriptSegmenter
import json

# Parse transcript từ timestamps
segmenter = TranscriptSegmenter(video_transcript)

# Segment theo outline
outlined_sections = segmenter.segment_by_outline(outline.sections)

print("===== SEGMENTED SECTIONS =====")
print(f"Total sections: {len(outlined_sections)}")
print(json.dumps(outlined_sections, indent=2, ensure_ascii=False))

===== SEGMENTED SECTIONS =====
Total sections: 38
[
  {
    "section_id": 1,
    "title": "Introduction to Dreams as Aspirations",
    "start": 0.0,
    "end": 33.0,
    "text": "hello everyone leo here ready to embark on a journey into the world of dreams not the sleepy time ones but those dreams that get you up in the morning and i'm gwen here to explore how our passions shape our lives"
  },
  {
    "section_id": 2,
    "title": "Defining and Pursuing Dreams",
    "start": 33.0,
    "end": 148.0,
    "text": "passions shape our lives so leo let's define dreams in the context of today's chat right gwen when we talk about dreams here we're talking about those big goals and aspirations the things that make us excited about the future like someone dreaming of becoming an artist an engineer or mastering a new skill it's all about what you're passionate about exactly and my dream it's to bring joy to people whether it's through comedy acting or just being my charming self i mean who would

In [6]:
# Summarize each section with memory

from llm.gemini_client import GeminiClient
from llm.prompts import build_section_summary_prompt
from schemas.output_format import SectionSummaryOutput


memory = ""
section_summaries = []

for section in outlined_sections:
    section_text = section["text"]

    prompt = build_section_summary_prompt(
        section_text=section_text,
        memory=memory,
        video_language=result["language"]
    )

    response = gemini.models.generate_content(
        model="models/gemini-2.5-flash-lite",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_json_schema": SectionSummaryOutput.model_json_schema(),
        }
    )

    # ---- Parse JSON output ----
    summary_obj = SectionSummaryOutput.model_validate_json(response.text)

    # ---- Lưu kết quả ----
    section_summaries.append({
        "section_id": section["section_id"],
        "title": section["title"],
        "start": section["start"],
        "end": section["end"],
        "summary": summary_obj.summary,
    })

    # ---- Update memory ----
    memory = summary_obj.summary

print(json.dumps(section_summaries, indent=2, ensure_ascii=False))


[
  {
    "section_id": 1,
    "title": "Introduction to Dreams as Aspirations",
    "start": 0.0,
    "end": 33.0,
    "summary": "Leo and Gwen introduce the topic of dreams, distinguishing between sleep-related dreams and life aspirations. They set the stage to explore how these passions influence our lives."
  },
  {
    "section_id": 2,
    "title": "Defining and Pursuing Dreams",
    "start": 33.0,
    "end": 148.0,
    "summary": "This section clarifies that dreams, in the context of the discussion, refer to significant goals and aspirations, encompassing passions like becoming an artist or engineer. Gwen emphasizes the joy derived from pursuing these dreams, while Leo highlights their role in making a difference. The speakers underscore that dreams are not solely about the end result but also about the process and the feeling of being alive. They use learning English as an example of a dream that opens up opportunities and requires courage and commitment, ultimately pushing indi

In [7]:
section_summaries_text = "" 
section_summaries_text = json.dumps(
    section_summaries,
    indent=2,
    ensure_ascii=False
)

In [8]:
print(section_summaries_text)

[
  {
    "section_id": 1,
    "title": "Introduction to Dreams as Aspirations",
    "start": 0.0,
    "end": 33.0,
    "summary": "Leo and Gwen introduce the topic of dreams, distinguishing between sleep-related dreams and life aspirations. They set the stage to explore how these passions influence our lives."
  },
  {
    "section_id": 2,
    "title": "Defining and Pursuing Dreams",
    "start": 33.0,
    "end": 148.0,
    "summary": "This section clarifies that dreams, in the context of the discussion, refer to significant goals and aspirations, encompassing passions like becoming an artist or engineer. Gwen emphasizes the joy derived from pursuing these dreams, while Leo highlights their role in making a difference. The speakers underscore that dreams are not solely about the end result but also about the process and the feeling of being alive. They use learning English as an example of a dream that opens up opportunities and requires courage and commitment, ultimately pushing indi

In [9]:
prompt = build_global_summary_prompt(
        section_summaries=section_summaries_text,
        video_language=result["language"]   
    )

NameError: name 'build_global_summary_prompt' is not defined

In [ ]:
# Global Summary

from llm.gemini_client import GeminiClient
from llm.prompts import build_global_summary_prompt
from schemas.output_format import GlobalSummaryOutput

# Prepare section summaries as input
section_summaries_text = "" 
for sec in section_summaries:
    section_summaries_text += f"Section {sec['section_id']} - {sec['title']}:\n{sec['summary']}\n\n"
prompt = build_global_summary_prompt(
    section_summaries=section_summaries_text,
    video_language=result["language"]   
)
response = gemini.models.generate_content(
    model="models/gemini-2.5-flash-lite",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_json_schema": GlobalSummaryOutput.model_json_schema(),
    }
)
overall_summary = GlobalSummaryOutput.model_validate_json(response.text)
print("===== GLOBAL SUMMARY =====")
print(json.dumps(overall_summary.model_dump(), indent=2, ensure_ascii=False))

===== GLOBAL SUMMARY =====
{
  "global_summary": "This video explores various aspects of personal growth and well-being, covering topics such as dreams and aspirations, the importance of learning, dealing with mistakes and failure, building confidence, managing stress, and adapting to new environments. It offers practical advice, vocabulary, and strategies for self-improvement and navigating life's challenges.",
  "section_takeaways": [
    {
      "section_id": 1,
      "title": "Introduction to Dreams and Aspirations",
      "start": 0.0,
      "end": 0.0,
      "takeaways": [
        "Dreams are goals driven by passion and enjoyment of the pursuit.",
        "Pursuing dreams requires courage and pushes personal growth.",
        "Dreams make life more exciting and rewarding."
      ]
    },
    {
      "section_id": 2,
      "title": "Dreams Vocabulary and Discussion Recap",
      "start": 0.0,
      "end": 0.0,
      "takeaways": [
        "Dreams are strong aspirations and ambitio